# legacy

## get and print all xml tags (in hierarchical order) used in a xml file

In [ ]:
import xml.etree.ElementTree as ET 
from pprint import pprint

In [ ]:
tree = ET.parse('filename')
root = tree.getroot()

elemList = []

for elem in tree.iter():
    elemList.append(elem.tag)

elemList = list(set(elemList))
pprint(elemList)

## parsing xml

- findall-method is rather slow
- simple iter should be fastest method but is limited in configuration options
- iterparse is slower than others due to overhead
- simple xpath (lxml) is best since it is both fast and easy to hdle

In [ ]:
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd
from pathlib import Path

In [ ]:
namespaces = {'atom': 'http://www.w3.org/2005/Atom', 'cei': 'http://www.monasterium.net/NS/cei'}
directoryPath = '../data/db/mom-data/metadata.charter.public/'
fileExtension = "*.cei.xml"

In [ ]:
lists = ["tenor_contents", "atom_IDs", "places", "languages"]
tenor_contents, atom_IDs, places, languages = ([] for i in range(len(lists)))

In [ ]:
#xpath lxml
for file in Path(directoryPath).rglob(fileExtension):
    tree = etree.parse(str(file))
    for tenor in tree.xpath("//cei:tenor", namespaces = namespaces):
        tenor_contents.append(tenor.text)
        atom_IDs.append(tree.xpath("//atom:id/text()", namespaces = namespaces))
        places.append(tree.xpath(".//cei:placeName/text()", namespaces = namespaces))
        languages.append(tree.xpath(".//cei:lang_MOM/text()", namespaces = namespaces))

In [ ]:
#findall etree

for file in Path(directoryPath).rglob(fileExtension):
    root = etree.parse(str(file)).getroot()
    for tenor in root.findall(".//cei:tenor", namespaces):
        tenor_contents.append(etree.tostring(tenor).decode("utf-8")) # diese Zeile killt den Speicher
        atom_IDs.append(root[0].text)
        if root.find('.//cei:placeName', namespaces) is not None:
            places.append(root.find('.//cei:placeName', namespaces).text)
        elif root.find('.//cei:placeName', namespaces) is None:
            places.append(None)
        if root.find('.//cei:lang_MOM', namespaces) is not None:
            languages.append(root.find('.//cei:lang_MOM', namespaces).text)
        elif root.find('.//cei:lang_MOM', namespaces) is None:
            languages.append(None)


       # https://stackoverflow.com/questions/12964996/xpath-normalize-space

        # one possible solution would be to make generators that are then linked, so linked lists for each item
        # for element in root.iter("{http://www.w3.org/2005/Atom}id"):
        #     atom_id.append(element.text)
        # for element in root.iter("{http://www.monasterium.net/NS/cei}graphic"):
        #     cei_graphic_ATTRIBUTE_url_copy.append(element.text)


## filter by image

In [ ]:
import pandas as pd

#df_links = df[df['url'].str.contains("http://")]
#df_links.to_csv(f'links_{pathname}.csv', index=False)

In [ ]:
df = pd.read_csv('../data/output/metadata.charter.public.csv')

In [ ]:
df

In [ ]:
df_withMomLinks = df[df['url'].str.contains("images.monasterium.net", na=False)]
df_withMomLinks.to_csv(f'../data/output/df_withMomLinks.csv', index=False)

In [ ]:
df_withMomLinks

### add filter for no url

In [ ]:
pathname = PurePath(directoryPath).name
df.to_csv(f'../data/output/{pathname}.csv', index=False)

df_noURLs = df[df['url'].astype(bool)]

df_noURLs.to_csv(f'../data/output/{pathname}_nourls.csv'
    f'{pathname}_noURLs.csv', index=False)

# transform referential img-.xml files (plural!) into csv using lxml (relatively fast)

In [ ]:
from pathlib import PurePath
from pathlib import Path
from lxml import etree
import pandas as pd

In [ ]:
namespaces = {'atom': 'http://www.w3.org/2005/Atom', 'cei': 'http://www.monasterium.net/NS/cei'}
directoryPath = '\\\?/'+'C://Users/atzenhof/playground/data/monasterium/MOM-data/image_xmls' # path escape due to long path
fileExtension = ('*.xml')

In [ ]:
atomIDs = []
image_links = []

for file in Path(directoryPath).rglob(fileExtension):
    tree = etree.parse(str(file)) # requres conversion to str since lxml does not vibe with windowspath
    root = tree.getroot()
    for img in root.findall('.//img', namespaces):
        atomID = img.getparent().attrib['id']
        atomIDs.append(atomID)
        image_link = img.attrib['src']
        image_links.append(image_link)

In [ ]:
img_list = list(zip(atomIDs, image_links))
df = pd.DataFrame(img_list).rename(columns={0: 'atom', 1: 'url'})
df

## .xml-file(s) on images [stemming from Georg's xquery] -> .csv ✔

In [ ]:
from pathlib import PurePath
from pathlib import Path
from lxml import etree
import pandas as pd

In [ ]:
namespaces = {"atom": "http://www.w3.org/2005/Atom", "cei": "http://www.monasterium.net/NS/cei"}
#directoryPath = "../data/output"
directoryPath = '\\\?/'+'C://Users/atzenhof/playground/GitHub/didip/data/images_xml' # path escape due to long path
fileExtension = ('*.xml')

In [ ]:
atomIDs = []
image_links = []

for file in Path(directoryPath).rglob(fileExtension):
    tree = etree.parse(str(file)) # requres conversion to str since lxml does not vibe with windowspath
    root = tree.getroot()
    for img in root.findall('//img', namespaces):
        atomID = img.getparent().attrib['id']
        atomIDs.append(atomID)
        image_link = img.attrib['src']
        image_links.append(image_link)

In [ ]:
img_list = list(zip(atomIDs, image_links))
df = pd.DataFrame(img_list).rename(columns={0: 'atomID', 1: 'url'})
df

In [ ]:
pathname = PurePath(directoryPath).name
df.to_csv(f'../data/output/test_{pathname}.csv', index=False)

## find lengths of lists

In [ ]:
# def find_max_list(list):
#     list_len = [len(i) for i in list]
#     maxvalue = max(list_len)
#     print(maxvalue)

# info_list_list = []
# for info_list in charter_image_list["atom_parts"]:
#     info_list_list.append(info_list)

# find_max_list(info_list_list) # yields 5, means 3 without first two parts is normal

# buggy way of checking url status

In [ ]:
#does not work well with ascii 

def get_url_status(url):
    if type(url) == str:
        url_clean = url.replace(" ", "%20")
        request = Request(url_clean, headers={"User-Agent": "Mozilla"})
        try:
            response = urlopen(request)
        except HTTPError as e:
            return(f"Failed at {datetime.datetime.now()}: Error {e.code}")
        except URLError as e:
            return(f"Failed to reach server at {datetime.datetime.now()}. Reason: {e.reason}")
        else:
            return(f"URL was up at {datetime.datetime.now()}")
    else:
        pass

In [ ]:
for url in df_archive_links_clean_test["charter_image_url"]:
    print(url)
    df_archive_links_clean_test["test_ping"] = get_url_status(url)

# checking url status

In [ ]:
number_of_downs = 0
list_of_downs = []
for url in series:
    if urllib.request.urlopen(url).getcode() == 200:
        print(f"{url} is up.")
    else:
        print(f"DOWN! - {url}")
        number_of_downs += 1
        list_of_downs.append(url)

# Queries

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/output/charter_contents_full_metadata.charter.public.csv', encoding="utf-8")

## Stuff

In [ ]:
new = df.groupby("cei_lang_MOM")["atom_id"].count()
new.sort_values(ascending=False)

In [ ]:
df_dt_tenor = (df.loc[df["cei_lang_MOM"] == "['dt.']"]).loc[df["cei_tenor"] != "[]"]
df_dt_pTenor = (df.loc[df["cei_lang_MOM"] == "['dt.']"]).loc[df["cei_pTenor"] != "[]"]

print(f"Charters with 'dt.' in cei_lang_MOM and containing a cei_tenor: {len(df_dt_tenor)}")
print(f"Charters with 'dt.' in cei_lang_MOM and containing a cei_pTenor: {len(df_dt_pTenor)}")

In [ ]:
#sorting, filtering, limiting etc
#df.loc[df['columnname'] == 'something']
#df.groupby('columnname')['countingtag'].count()
#df.sort_values(ascending=False)

In [ ]:
# apply filter based on cell
df['url'] = df['url'].apply(lambda x : x[1:] if x.startswith("/") else x) #exclude / at start for x , fastest method

In [ ]:
# check if cell is empty list
test = df[df["tag"].astype(str) != '[]']

In [ ]:
# reset index and groupby
charter_contents_exploded.reset_index(drop=True, inplace=True)
charter_contents_exploded.groupby('cei_lang_MOM')['atom_id'].count().sort_values(ascending=False).head(n=20)

# filters

In [ ]:
df_text = contents_full[contents_full["cei_tenor"].str.len() >= 30]
df_german = df_text[df_text["cei_lang_MOM"].astype(str).str.contains("Deutsch", "dt.")]
df_no_latin = df_german[~df_german["cei_lang_MOM"].astype(str).str.contains("Latein", "lt.")]
df_no_latin2 = df_no_latin[~df_no_latin["cei_lang_MOM"].astype(str).str.contains("lateinisch")]
df_AT = df_no_latin2[df_no_latin2["atom_id"].astype(str).str.contains("/AT-")]
df_time = df_AT.explode("cei_dateRange_ATTRIBUTE_from").explode("cei_dateRange_ATTRIBUTE_to").explode("cei_date_ATTRIBUTE_value")
df_mhd_top = df_time[df_time["cei_dateRange_ATTRIBUTE_from"] <= "14000000"]
df_mhd_bot = df_mhd_top[df_mhd_top["cei_dateRange_ATTRIBUTE_from"] >= "11000000"]
df_sorted = df_mhd_bot.sort_values("cei_dateRange_ATTRIBUTE_from" , ascending=True)

In [ ]:
# Comparisons, checks in dataframe
df.loc[(df['Space'] == 'TRUE') & (df['Threshold'] <= 0.2), 'Space_Test'] = 'Pass'
df.loc[(df['Space'] != 'TRUE') | (df['Threshold'] > 0.2), 'Space_Test'] = 'Fail'


df['Space_Test'] = np.where(df['Space'],
                   np.where(df['Threshold'] <= 0.2, 'Pass', 'Fail'),'FALSE')

# places?

In [ ]:
charters_w_places = contents_full[contents_full["cei_placeName"].astype(str) != '[]']
len(charters_w_places)